*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

# Model Zoo -- Perceptron

In [1]:
import tensorflow as tf
import numpy as np
from io import BytesIO


##########################
### DATASET
##########################

ds = np.lib.DataSource()
fp = ds.open('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data')

x = np.genfromtxt(BytesIO(fp.read().encode()), delimiter=',', usecols=range(4), max_rows=100)
y = np.zeros(100)
y[50:] = 1


##########################
### SETTINGS
##########################

n_features = x.shape[1]
n_samples = x.shape[0]
training_epochs = 5


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default() as g:

   # initialize model parameters
    tf_x = tf.placeholder(dtype=tf.float32,
                          shape=[None, n_features], name='features')
    tf_y = tf.placeholder(dtype=tf.float32,
                          shape=[None, 1], name='targets')
    params = {
        'weights': tf.Variable(tf.zeros(shape=[n_features, 1],
                                        dtype=tf.float32), name='weights'),
        'bias': tf.Variable([[0.]], dtype=tf.float32, name='bias')}

    # forward pass
    linear = tf.matmul(tf_x, params['weights']) + params['bias']
    ones = tf.ones(shape=tf.shape(linear))
    zeros = tf.zeros(shape=tf.shape(linear))
    prediction = tf.where(tf.less(linear, 0.), zeros, ones, name='prediction')

    # weight update
    diff = tf_y - prediction

    weight_update = tf.assign_add(params['weights'],
                                  tf.reshape(diff * tf_x,
                                             (n_features, 1)))

    # workaround to update both weights and bias via one operation
    with tf.control_dependencies([weight_update]):
        train = tf.assign_add(params['bias'], diff, name='train')


##########################
### TRAINING & EVALUATION
##########################
    
        
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        for xi, yi in zip(x, y):
            feed_dict = {'features:0': xi.reshape(-1, n_features),
                         'targets:0': yi.reshape(-1, 1)}
            _ = sess.run('train:0', feed_dict=feed_dict)
    
        prediction = sess.run('prediction:0', feed_dict={'features:0': x})
        train_error = np.sum(prediction.reshape(-1) != y) / n_samples
        print('Epoch %02d | Training Error: %.2f' % (epoch + 1, train_error))
    
    weights, bias = sess.run(['weights:0', 'bias:0'])
    print('\nWeights:\n', weights)
    print('\nBias:\n', bias)

Epoch 01 | Training Error: 0.50
Epoch 02 | Training Error: 0.50
Epoch 03 | Training Error: 0.00
Epoch 04 | Training Error: 0.00
Epoch 05 | Training Error: 0.00

Weights:
 [[-1.29999971]
 [-4.0999999 ]
 [ 5.19999933]
 [ 2.19999981]]

Bias:
 [[-1.]]
